In [1]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=8, app_name='west0_ws_seg3', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4047 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_ws_seg3...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:23


In [23]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [3]:
# 각 day 28
start_date = "2022-06-08"
end_date = "2022-07-05"

ws_start_date = "2022-07-13"
ws_day_28 = "2022-08-09"

In [4]:
gcoin = load_data_mart("pc", ws_start_date, ws_day_28, "gcoin_use")
wsus_gcoin = gcoin.where(col("event_name").like("%wsus%"))
wsus_user = wsus_gcoin.select("account_id").distinct()
costume = gcoin.where(col("event_name").like("%workshop%"))
costume_user = costume.select("account_id").distinct()

In [5]:
only_costume_gcoin = costume.join(wsus_user, "account_id", "leftanti")
only_wsus_gcoin= wsus_gcoin.join(costume_user, "account_id", "leftanti")

In [6]:
costume_and_wsus_gcoin_user = wsus_gcoin.join(costume_user, "account_id").select("account_id").distinct()

In [7]:
costume_and_wsus_gcoin = costume.select("account_id", "product_id", "paid_use", "free_use", "sub_category", "event_type", "event_name", "price") \
    .unionByName(wsus_gcoin.select("account_id", "product_id", "paid_use", "free_use", "sub_category", "event_type", "event_name", "price")) \
    .join(costume_and_wsus_gcoin_user, "account_id")

## a

In [8]:
from pyspark.sql.types import * 

def chest_cnt(product_id):
    if product_id in ["itemdesc.14300001", "itemdesc.14300002", "itemdesc.13000577", "itemdesc.13000580"]:
        return 1
    elif product_id in ["itemdesc.13000578", "itemdesc.13000581"]:
        return 5
    elif product_id == "itemdesc.13000582":
        return 10
    else:
        return 0

assign_chest_cnt = udf(chest_cnt, IntegerType())
only_costume_gcoin = only_costume_gcoin.withColumn("chest_cnt", assign_chest_cnt("product_id"))

In [9]:
only_costume_gcoin_by_user = only_costume_gcoin.groupBy("account_id") \
    .agg(sum(col("free_use") + col("paid_use")).alias("total_gcoin"), sum("paid_use").alias("paid_use"), sum("chest_cnt").alias("chest_cnt"))

In [10]:
only_costume_gcoin_by_user = only_costume_gcoin_by_user.withColumn("group", \
                                         when(col("total_gcoin") <= 1000, "A") \
                                        .when((col("total_gcoin") > 1000) & (col("total_gcoin") <= 5000), "B") \
                                        .when((col("total_gcoin") > 5000) & (col("total_gcoin") <= 10000), "C") \
                                        .when((col("total_gcoin") > 10000) & (col("total_gcoin") <= 30000), "D") \
                                        .when((col("total_gcoin") > 30000), "E") \
                                     )
only_costume_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_gcoin").alias("total_gcoin"), sum("paid_use").alias("paid_use"), sum("chest_cnt").alias("chest_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_gcoin,paid_use,chest_cnt
0,A,389679,119843140,20753695,445366
1,B,111044,232457230,121828170,881454
2,C,21808,152104690,126634285,569479
3,D,21504,363651570,319809125,1236826
4,E,8757,527643200,471453105,1365874


In [11]:
only_costume_gcoin.join(only_costume_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").otherwise(lit("costume"))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("user_cnt"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("paid_use").alias("paid_use"), sum("chest_cnt").alias("chest_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,user_cnt,total_use,paid_use,chest_cnt
0,A,workshop,389679,119843140,20753695,445366
1,B,workshop,111044,232457230,121828170,881454
2,C,workshop,21808,152104690,126634285,569479
3,D,workshop,21504,363651570,319809125,1236826
4,E,workshop,8757,527643200,471453105,1365874


In [12]:
only_costume_gcoin_by_user = only_costume_gcoin.join(only_costume_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").otherwise(lit("costume"))) \
    .groupBy("type", "account_id", "group").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use")+col("free_use")).alias("total_use"), sum("chest_cnt").alias("chest_cnt")).toPandas()
only_costume_gcoin_by_user.to_csv("./only_costume_gcoin_by_user3_day28.csv", index=False)

## b

In [13]:
def wsus_cnt(price):
    if price == 1800:
        return 10
    elif price == 200:
        return 1
    else:
        return 0
    
assign_wsus_cnt = udf(wsus_cnt, IntegerType())
costume_and_wsus_gcoin = costume_and_wsus_gcoin.withColumn("wsus_cnt", assign_wsus_cnt("price"))

In [14]:
assign_chest_cnt = udf(chest_cnt, IntegerType())
costume_and_wsus_gcoin = costume_and_wsus_gcoin.withColumn("chest_cnt", assign_chest_cnt("product_id"))

In [15]:
costume_and_wsus_gcoin_by_user = costume_and_wsus_gcoin.groupBy("account_id") \
    .agg(sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))

In [16]:
costume_and_wsus_gcoin_by_user = costume_and_wsus_gcoin_by_user.withColumn("group", \
                                                                when(col("wsus_cnt") <=3, "A") \
                                                                .when((col("wsus_cnt") >= 4) & (col("wsus_cnt") <= 15), "B") \
                                                                .when((col("wsus_cnt") >= 16) & (col("wsus_cnt") <= 30), "C") \
                                                                .when((col("wsus_cnt") >= 31) & (col("wsus_cnt") <= 240), "D") \
                                                                .when((col("wsus_cnt") >= 241), "E") \
                                                                          )
costume_and_wsus_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_use").alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_use,paid_use,wsus_cnt,chest_cnt
0,A,63025,170277570,121538375,97884,505217
1,B,29563,229687250,182206960,250054,584399
2,C,10711,178131830,154328580,246352,406496
3,D,18773,674328360,597459870,1597999,1087851
4,E,2450,302897860,271053010,1148176,229509


In [17]:
costume_and_wsus_gcoin.join(costume_and_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").when(col("event_name").like("%wsus%"), "wsus").otherwise(lit("costume"))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("user_cnt"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,user_cnt,total_use,paid_use,wsus_cnt,chest_cnt
0,A,workshop,63025,150700770,118176390,0,505217
1,A,wsus,63025,19576800,3361985,97884,0
2,B,workshop,29563,182467450,157164050,0,584399
3,B,wsus,29563,47219800,25042910,250054,0
4,C,workshop,10711,132934230,116752785,0,406496
5,C,wsus,10711,45197600,37575795,246352,0
6,D,workshop,18773,384190160,338452020,0,1087851
7,D,wsus,18773,290138200,259007850,1597999,0
8,E,workshop,2450,95613260,83815050,0,229509
9,E,wsus,2450,207284600,187237960,1148176,0


In [18]:
costume_and_wsus_gcoin_by_user2 = costume_and_wsus_gcoin.join(costume_and_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").when(col("event_name").like("%wsus%"), "wsus").otherwise(lit("costume"))) \
    .groupBy("type", "account_id", "group").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use")+col("free_use")).alias("total_use"), sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt")).toPandas()
costume_and_wsus_gcoin_by_user2.to_csv("./costume_and_wsus_gcoin_by_user3_day28.csv", index=False)

## c

In [19]:
only_wsus_gcoin = only_wsus_gcoin.withColumn("wsus_cnt", assign_wsus_cnt("price"))

In [20]:
only_wsus_gcoin_by_user = only_wsus_gcoin.groupBy("account_id") \
    .agg(sum("wsus_cnt").alias("wsus_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))

In [21]:
only_wsus_gcoin_by_user = only_wsus_gcoin_by_user.withColumn("group", \
                                                when(col("wsus_cnt") <=10, "A") \
                                                .when((col("wsus_cnt") >= 11) & (col("wsus_cnt") <= 50), "B") \
                                                .when((col("wsus_cnt") >= 51) & (col("wsus_cnt") <= 100), "C") \
                                                .when((col("wsus_cnt") >= 101) & (col("wsus_cnt") <= 800), "D") \
                                                .when((col("wsus_cnt") >= 801), "E") \
                                                          )
only_wsus_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_use").alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_use,paid_use,wsus_cnt
0,A,286793,183725800,13841320,955974
1,B,16465,65373200,48192640,355905
2,C,2998,38094000,33293205,210074
3,D,2328,86607800,77313675,479363
4,E,59,11034800,9865030,61249


In [22]:
only_wsus_gcoin_by_user.toPandas().to_csv("./only_wsus_gcoin_by_user3_day28.csv", index=False)